In [1]:
import torch
from transformers import (
    BitsAndBytesConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    pipeline
)
from peft import (
    PeftModel,
    PeftConfig
)
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/ubuntu/miniconda3/envs/test/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
FINETUNED_MODEL = "./pcp_qlora"

peft_config = PeftConfig.from_pretrained(FINETUNED_MODEL)

In [3]:
# NF4 양자화를 위한 설정
nf4_config = BitsAndBytesConfig(
    # load_in_4bit=True, # 모델을 4비트 정밀도로 로드
    load_in_8bit=True, # 모델 학습 시간 및 gpu 자원 제한으로 8비트 로드
    bnb_4bit_quant_type="nf4", # 4비트 NormalFloat 양자화: 양자화된 파라미터의 분포 범위를 정규분포 내로 억제하여 정밀도 저하 방지
    bnb_4bit_use_double_quant=True, # 이중 양자화: 양자화를 적용하는 정수에 대해서도 양자화 적용
    bnb_4bit_compute_dtype=torch.bfloat16 # 연산 속도를 높이기 위해 사용 (default: torch.float32)
)

# # 베이스 모델 및 토크나이저 로드
# model = AutoModelForCausalLM.from_pretrained(
#     peft_config.base_model_name_or_path,
#     quantization_config=nf4_config,
#     device_map="auto",
#     torch_dtype=torch.bfloat16
# )
# tokenizer = AutoTokenizer.from_pretrained(
#     peft_config.base_model_name_or_path
# )

# 베이스 모델 및 토크나이저 로드
model = AutoModelForCausalLM.from_pretrained(
    "MLP-KTLim/llama-3-Korean-Bllossom-8B",
    quantization_config=nf4_config,
    device_map="auto",
    torch_dtype=torch.bfloat16
)
tokenizer = AutoTokenizer.from_pretrained(
    "MLP-KTLim/llama-3-Korean-Bllossom-8B"
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:08<00:00,  2.24s/it]


In [4]:
# QLoRA 모델 로드
peft_model = PeftModel.from_pretrained(model, FINETUNED_MODEL, torch_dtype=torch.bfloat16)

In [5]:
# QLoRA 가중치를 베이스 모델에 병합
merged_model = peft_model.merge_and_unload()

/home/ubuntu/miniconda3/envs/test/lib/python3.10/site-packages/peft/tuners/lora/bnb.py:83: UserWarning: Merge lora module to 8-bit linear may get different generations due to rounding errors.
  warnings.warn(


In [8]:
starter = torch.cuda.Event(enable_timing=True)
ender = torch.cuda.Event(enable_timing=True)

prompt = "여보세요?"

# 텍스트 생성을 위한 파이프라인 설정
with torch.no_grad():
    starter.record()

    pipe = pipeline("text-generation", model=merged_model, tokenizer=tokenizer, max_new_tokens=256)
    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.92,
        repetition_penalty=1.2,
        add_special_tokens=True 
    )
    res = outputs[0]["generated_text"][len(prompt):]

    ender.record()

torch.cuda.synchronize()
inference_time = starter.elapsed_time(ender)

print(f"Elapsed time: {round(inference_time * 1e-3, 2)} s\nResult: {res.replace('<|im_end|>', '')}")

/home/ubuntu/miniconda3/envs/test/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Elapsed time: 2.38 s
Result:  요즘 어떻게 지내시나요?
